In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    SentinelHubStatistical,
    SHConfig,
)

In [ ]:
# Only run this cell if you have not created a configuration.
import os

config = SHConfig()
config.sh_client_id = os.getenv("SH_CLIENT_ID")
config.sh_client_secret = os.getenv("SH_CLIENT_SECRET")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"

In [ ]:
bbox = BBox((73.795987,18.611295,73.830256,18.639324), crs=CRS.WGS84).transform(3035)
time_interval = "2023-04-01", "2023-09-01"
resolution = (50, 50)

In [ ]:
evalscript_bare_soil = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B04", "B08", "B11", "B12", "SCL", "dataMask"],
    output: { bands: 3, sampleType: "UINT16" },
    mosaicking: "ORBIT",
  };
}

function isCloud(sample) {
  // Define codes as invalid:
  const invalid = [
    0, // NO_DATA
    1, // SATURATED_DEFECTIVE
    3, // CLOUD_SHADOW
    7, // CLOUD_LOW_PROBA
    8, // CLOUD_MEDIUM_PROBA
    9, // CLOUD_HIGH_PROBA
    10, // THIN_CIRRUS
  ];
  return !invalid.includes(sample.SCL);
}

function evaluatePixel(samples) {
  let [nBare, nNonBare, nCloud] = [0, 0, 0];
  for (let i = 0; i < samples.length; i++) {
    let s = samples[i];
    if (s.dataMask === 0) {
      continue;
    }
    if (isCloud(s)) {
      nCloud++;
      continue;
    }
    // bareness index
    let mbi =
      (2.5 * (s.B11 + s.B04 - (s.B08 + s.B02))) /
      (s.B11 + s.B04 + (s.B08 + s.B02));
    if (mbi > 0) {
      nBare++;
    } else {
      nNonBare++;
    }
  }
  return [nBare, nNonBare, nCloud];
}

"""

In [ ]:
request = SentinelHubRequest(
    evalscript=evalscript_bare_soil,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                "s2l2a", service_url=config.sh_base_url
            ),
            time_interval=time_interval,
            mosaicking_order=MosaickingOrder.MOST_RECENT,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=bbox,
    resolution=resolution,
    config=config,
    data_folder="./data",
)

In [ ]:
bare_soil = request.get_data(save_data=True)[0]

In [ ]:
total_clear = bare_soil[:, :, 1] + bare_soil[:, :, 0]
bare_ratio = bare_soil[:, :, 0] / total_clear

# show image of bare soil ratio for the area of interest.

plt.figure(figsize=(24, 24))
plt.imshow(bare_ratio, cmap="cividis");

In [2]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import os

# Your client credentials
client_id = os.getenv("SH_CLIENT_ID")
client_secret = os.getenv("SH_CLIENT_SECRET")

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret, include_client_id=True)

# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
print(resp.content)

b'[]'


In [23]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: { bands: 3 },
  }
}

function evaluatePixel(sample) { 
    return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02]
}
"""

request={
    "input": {
        "bounds": {
            "properties": {
                "crs": "http://www.opengis.net/def/crs/EPSG/0/32633"
            },
            "bbox": [
                73.795987, 18.611295, 73.830256, 18.639324
            ]
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "timeRange": {
                        "from": "2024-01-01T00:00:00Z",
                        "to": "2024-08-01T00:00:00Z"
                    }
                }
            }
        ]
    },
    "output": {
        "width": 256,
        "height": 256
    },
    "evalscript": evalscript
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)
if response.status_code == 200:
        # Save the PNG image
        with open("latest_satellite_image.png", "wb") as f:
            f.write(response.content)
        print("Image saved successfully as 'latest_satellite_image.png'.")
else:
    print(f"Failed to retrieve image. Status code: {response.status_code}")
    print("Response:", response.text)

Image saved successfully as 'latest_satellite_image.png'.


In [19]:
import requests
from datetime import datetime, timezone

# Set up your bounding box and collections to check
bbox = [72.708549,19.041673,72.747002,19.083047]
data_collections = ["sentinel-2-l2a", "sentinel-1-grd", "sentinel-3-olci"]
crs = "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
from_date = "2019-01-01T00:00:00Z"
to_date = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")  # Current time

latest_images = []

for collection in data_collections:
    request_payload = {
        "bbox": bbox,
        "datetime": f"{from_date}/{to_date}",
        "collections": [collection],
        "limit": 1,  # Fetch only the latest image
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/search"
    response = oauth.post(url, json=request_payload)

    if response.status_code == 200:
        catalog_data = response.json()
        if catalog_data.get('features'):
            latest_image = catalog_data['features'][0]
            latest_date = latest_image['properties']['datetime']
            cloud_coverage = latest_image['properties'].get('eo:cloud_cover', 'N/A')
            print(latest_image)
            latest_images.append({
                "collection": collection,
                "date": latest_date,
                "cloud_coverage": cloud_coverage
            })
            print(f"Latest image for {collection}: {latest_date} with cloud coverage: {cloud_coverage}%")
        else:
            print(f"No images found for {collection} in the given date range.")
    else:
        print(f"Failed to query {collection}. Status code: {response.status_code}")
        print("Response:", response.text)

{'stac_version': '1.0.0', 'stac_extensions': ['https://stac-extensions.github.io/eo/v1.0.0/schema.json', 'https://stac-extensions.github.io/projection/v1.0.0/schema.json'], 'id': 'S2B_MSIL2A_20240908T053649_N0511_R005_T42QZG_20240908T100127.SAFE', 'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC::CRS84'}}, 'coordinates': [[[[71.86453529022592, 19.87336816393549], [72.91142127555993, 19.85344385188375], [72.8878518821865, 18.863472484408675], [71.84724952790602, 18.882328259044897], [71.86453529022592, 19.87336816393549]]]]}, 'bbox': [71.84724952790602, 18.863472484408675, 72.91142127555993, 19.87336816393549], 'properties': {'datetime': '2024-09-08T05:53:44.432Z', 'platform': 'sentinel-2b', 'instruments': ['msi'], 'constellation': 'sentinel-2', 'gsd': 10, 'eo:cloud_cover': 17.21, 'proj:epsg': 32642, 'proj:bbox': [799980.0, 2090220.0, 909780.0, 2200020.0], 'proj:geometry': {'type': 'MultiPolygon', 'crs': {'type':

In [8]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["VV"],
    output: { id: "default", bands: 1 },
  }
}

function evaluatePixel(samples) {
  return [2 * samples.VV]
}
"""

request = {
    "input": {
        "bounds": {
            "bbox": [
                72.852402,18.926748,72.881241,18.965065
            ],
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/3857"},
        },
        "data": [
            {
                "type": "sentinel-1-grd",
                "dataFilter": {
                    "timeRange": {
                        "from": "2024-01-01T00:00:00Z",
                        "to": "2024-01-05T23:59:59Z",
                    }
                },
                "processing": {
                    "orthorectify": "true",
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [
            {
                "identifier": "default",
                "format": {"type": "image/png"},
            }
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)
if response.status_code == 200:
    # Save the PNG image
    with open("latest_satellite_image.png", "wb") as f:
        f.write(response.content)
    print("Image saved successfully as 'latest_satellite_image.png'.")
else:
    print(f"Failed to retrieve image. Status code: {response.status_code}")
    print("Response:", response.text)


Image saved successfully as 'latest_satellite_image.png'.


In [10]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["VV"],
    output: { id: "default", bands: 1 },
  }
}

function evaluatePixel(samples) {
  return [2 * samples.VV]
}
"""

request = {
    "input": {
        "bounds": {
            "bbox": [
                72.421875,19.012787,72.457581,19.058224
            ],
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
        },
        "data": [
            {
                "type": "sentinel-1-grd",
                "dataFilter": {
                    "timeRange": {
                        "from": "2019-11-05T00:00:00Z",
                        "to": "2024-08-31T23:59:59Z",
                    },
                    "mosaickingOrder": "mostRecent",
                },
            }
        ],
    },
    "output": {
        "width": 256,
        "height": 256,
        "responses": [
            {
                "identifier": "default",
                "format": {"type": "image/png"},
            }
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)
if response.status_code == 200:
    # Save the PNG image
    with open("latest_satellite_image.png", "wb") as f:
        f.write(response.content)
    print("Image saved successfully as 'latest_satellite_image.png'.")
else:
    print(f"Failed to retrieve image. Status code: {response.status_code}")
    print("Response:", response.text)

Image saved successfully as 'latest_satellite_image.png'.


In [ ]:
response = oauth.get("https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-2-l2a/items/S2A_MSIL2A_20240831T052641_N0511_R105_T43QCA_20240831T112555.SAFE")
response.json()

In [3]:
%pip install mechanize cookiejar

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Created wheel for pager: filename=pager-3.3-py3-none-any.whl size=9113 sha256=bfa0a038292337902f1446a5ab649bdc20f610f46410dfb858d9875ba07a0610
  Stored in directory: /Users/adityakhalkar/Library/Caches/pip/wheels/3c/2c/fa/5e767d6b9f1beb706354afe9ee67056dadffc4f44e5bb10c61
Successfully built pager
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install mechanicalsoup

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 5.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%uvicorn` not found.


In [17]:
import requests

# Initialize a session to maintain cookies between requests
session = requests.Session()

# Set headers to mimic the browser
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-IN,en-GB;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
    "Host": "www.marinetraffic.com",
    "Referer": "https://www.marinetraffic.com/en/data/?asset_type=vessels&columns=flag%2Cshipname%2Cphoto...",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.6 Safari/605.1.15",
    "X-Requested-With": "XMLHttpRequest"
}

# Set cookies from your browser session
cookies = {
    '__cf_bmn': '4b5Y6ZHP9epYccH5gLodiRR7hpVeVlb6XUYUbwt0Tg-1726427512-1.0.1.1-bqn9kGKyACsoUezAniLh5fsb6B9MZPzHhrb6ffinHWLkiwcP5yGm.BaF0uuvmzNxRJkkmHvd2zoWNDEo0e7PBA',
    '_cfuvid': 'i.s1N5n0lNFZhqnlpOVhaFLVsQ1HR.5hsZQ0xfiVrN4-1726421614060-0.0.1.1-604800000',
    '_ga': 'GA1.2.161278673.1726393946',
    '_gid': 'GA1.2.630983661.1726393946',
    'AUTHEMAIL': 'testershinobiai@gmail.com&CHALLENGE=AySm3olaMk92Iv9Jp907',
    'authTokens': '%7B%22access_token%22%3A%22eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJqdGkiOiIxNzI2NDI2ODY3Ljc2NjQuNTc1NGZmMjliY2Q5ZmQwYzY5MjQiLCJpc3MiOiJhdXRoLm1hcmluZXRyYWZmaWMuY29tIiwiYXVkIjoibWFyaW5ldHJhZmZpYy5jb20iLCJpYXQiOjE3MjY0MjY4NjcsIm5iZiI6MTcyNjQyNjg2NywiZXhwIjoxNzI2NDI4MDY3LCJ1c2VySWQiOjcyMzYzNzYsImltcGVyc29uYXRvcklkIjpudWxsfQ.XwotY1MmvYI1eHvFTmnKrx8qmu22hSNs0HElFeQcmlK1Up-23eENukYYb4shtnmYTo6Q-ntbAk7bWurW_jozOU5HAqKdHdF2NNO_0pNU470l_L6t2tWCordM4Gu63nYlF3EzoJrQ8Ybob2jlc9FK07mUAeqDSLfNPwR5vcM6MhB87AKR55O-wVEDeZlK4PgLIlis93LCB-2Ktd1xqQuCcrs7ULwlfKnYAF_cuH04dk3u8jg0MdkZO2ZeXnVqTqHqOeukasgTMD-dePGkT-kvyjuQQP9D9xFiS8mHhbIvl1nDUFUYga7tHOFl5A0T_dhbMgrwT8D_8Fkg79iN42eYby46WjiJwd4DF_9793WCImILWKXEY2YLhBbdWtuj6TL2XuYyl-RalrCXE7duyfJdzmeniCK8tzGGyjylR5b7yLorGWfMK8dxjhuAF-rMZ58ErCoYrAevuSXKkQPUDN71PhcYsU_vRuYK3aPMq5GSyl_AhZJYaFQp1q8hXIcaQ2drNOtXy-W-MiUhRADj1VWN8rciM-kP5E0Itv5yue1hEDGOrdtXlV1BGuTjOJcAi4iT8R6baaNecf3hp_drKl1SqimPToHMNdZirYDjRIyB2_rQD8Uy4th9qoZdDeDex62sjP1VApihxMvoiHrPrULK7Mgydd-3Usg_8X-hgjaVaLI%22%2C%22refresh_token%22%3A%221QhoKY46n8w6RrT3tAqu65sJQcF6LMlJGGOC89W48kf3GVJZZkit9xglrHeVipLy6AKd40iYb2w6LiWAQbhabVotmbC3phixWX6ah3F0q4t5EYfd8gtJV4xBpC7ww1V4nRi3lfCgFilIled5VrpwdG6A743hiiyBx1iaa1jq3b6JzuUL6oLmDp5GKBTwkoMDVFmkUsWwO9s1ixQDpCJEk4LLOXuES4eHcyxTbu3gEXSjtCykChUomsNPGyjVTqDy3QUi05qmM4e3zQV77uvQJFMr823TbA7scJPvoNXc51fWMAhgF7NwY1Cg0k9Uy4j2qwjMRHoHiatSY8zOrJ2SNWXNFgZkOTkeEbDbtGYDoCd3F5HYW9bguupyY0xjQv9vt90EDB5PtdrGXglv07XziyCbXQ1PBNaB5mOE9cHaVHdPigogsQwPC6c2LjcS0SpdbHOCZUAppclauyviuHDUS2SE1VnYxs2mGje2EYp8uAMMOP62qTrbFwIazqGBwtnr',
    'authTokensExp': '1727722867',
    'CAKEPHP': '8i9o4hb8vb2eugm4sr1snl212e',
    'cf_clearance': 'Ba69w05NGEvdGbC6WaElx.GLhcoBoip1WFHUdtmVfOU-1726427614-1.2.1.1-IyVu5BUvivXwStVJ_GVPlZxEbNM7ryFuvBu.OBizn0P64KXLDxedGbXtXKQytnKgyV6xniC0QnyYSM0.8.E_OlA8GFB420NZJSIr9kY6p19iHaHMIxgzIpG3.iQXfpzFDBZwyVmG8DDvhDBuiLs0IoM7OPZlWKnju13v.7CUszyBlk.MG03E7GVi7j_F56IGz8HCqNmFk..MR1kSZldHlFFCrjwR_OOjZ8XU5nrbfcRFhmpSwNMDb0hqWMnAZiWUj7cQhwqKN6j.NJwVTrTbIduSajQocXvXcdH5w2Mbo8Evf70chbbx_wu079qLkoKTtVxbVpWv2q5mRAMH683HewZsyIkVpY2xr4OYO8ljj3Jj4SiFtlxMTCwpKH5.W3OyLneTym2sjbkg7200jrWpGw',
    'intercom-device-id-y4oe5nh8': '7701b2f0-8176-488b-8920-cdcef0625457',
    'intercom-id-y4oe5nh8': '4e8699b3-5900-46c3-8a58-f3ae33e19495',
    'intercom-session-y4oe5nh8': '',
    'mt_user[UserID]': '7236376',
    'SERVERID': 'app8nzs',
    'vTo1': ''
}

# Perform the request with cookies and headers
response = session.get("https://www.marinetraffic.com/en/reports/?asset_type=vessels&columns=flag,shipname,photo,recognized_next_port,reported_eta,reported_destination,current_port,imo,ship_type,show_on_live_map,time_of_latest_position,lat_of_latest_position,lon_of_latest_position,speed,notes&ship_type_in=8&recognized_next_port_in=2341", headers=headers, cookies=cookies)

# Check the response
if response.status_code == 200:
    print(response.json())
else:
    print(f"Request failed with status code: {response.status_code}")


Request failed with status code: 403
